In [12]:
#!pip install playwright
#!pip install geonamescache
import tweepy
import numpy as np
import pandas as pd
import json
from bs4 import BeautifulSoup
from helper import *
from IPython.display import clear_output #clears output
import datetime
import time
DRIVER = webdriver.Chrome()#the chrome

API_Key ='irXxqVrXin1KsDRM9g6HdF8p8'
API_Key_Secret = 'sLQvb8voyrTDz5S93vlt5jE5RlMi8xBuOQtSq2pgERe2QVawIC'

Bearer_Token = 'AAAAAAAAAAAAAAAAAAAAACsVlAEAAAAADib49u9Asa6Hbts8UwBEdtc0dsk%3D1d8lVaySeRg1CpJQ6HDLSwK6Mj9wJiptj49PAdwMuvW95zJoar'

Access_Token = '1122492746210660352-COptyz3RtF1AVokHw3027iOdbsZZl1'
Access_Token_Secret = 'slfBz8pcgCrUmewZgSgVKsmIFhfCcFuinAx16q5eh1Vvr'

In [13]:
# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(API_Key, API_Key_Secret)
# set access to user's access key and access secret
auth.set_access_token(Access_Token, Access_Token_Secret)
# calling the api
api = tweepy.API(auth)

Client = tweepy.Client(bearer_token=Bearer_Token) 



In [14]:
#tweets_list = api.search_30_day(label = '30day',query = '#blender', fromDate = FROM_DATE,toDate = TO_DATE,maxResults = 10,tweet_mode = "extended")
#tweets_list = tweepy.Cursor(api.search_tweets, "biden", count=100).items(100)


start_date = datetime.datetime.now() - datetime.timedelta(days=14)
end_date = start_date + datetime.timedelta(days=7)

# Search for tweets within the target date range

ALL_pages = tweepy.Cursor(api.search_tweets, q="since:{} until:{}".format(start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")), lang="en", tweet_mode="extended", count = 100).pages(50)


tweets_list = []

for page in ALL_pages:
    tweets_list.extend(page)

tweets_list = [t for t in tweets_list if t.in_reply_to_status_id == None]

In [17]:
#search_tweets(q, *, geocode, lang, locale, result_type, count, until, since_id, max_id, include_entities)

#user data
user_name = [];
location = [];
age = [];
gender = [];
followers = [];
following = [];
verified = []
joined = [] #when they joined
tweet_count_user = []

#tweet data
tweet = [];
word_count = [];
lang = []
likes = []
retweet_count = []
is_quote_status = []
date = []

trending_for_area = []

users = [];


for t in tweets_list:
    tweet.append(t.full_text);
    
    users.append(t.user);
    word_count.append(countWordsInString(t.full_text))
    
    lang.append(t.lang)
    likes.append(t.favorite_count) 
    retweet_count.append(t.retweet_count)
    is_quote_status.append(t.is_quote_status)
    date.append(t.created_at.isoformat()[:10])
    
i = 0
for x in users:
    if i%25 == 0:
        print("waiting")
        time.sleep(6)
    
    j = x._json
    #opens the page and waits for it to load
    screen_name = j['screen_name']
    country = check_country_real(j['location'])
    
    
    DRIVER.get('https://twitter.com/' + screen_name) 
    
    #get the easy stuff
    user_name.append(screen_name)
    following.append(j['friends_count']);
    followers.append(j['followers_count']);
    location.append(country);
    joined.append(x.created_at.isoformat()[:10])
    tweet_count_user.append(x.statuses_count)
    
    trnd = 0
    try:
        trnd = getTrendingTopics(api,country)
    except:
        trnd = []
    
    trending_for_area.append(trnd)
    verified.append(x.verified)
    
    
    
    tmp = getAge(DRIVER)#gets the age
    if(tmp == 2023 - x.created_at.year): #checks if the age is the same as the accounts age
        age.append(np.nan)
    else:
        age.append(tmp)
    
    
    gender.append(getGender(j['description'],j['location'],j['name']));
    
    ##this part simply displays at the bottom how many profiles were checked
    clear_output(wait=True) #clears the output
    
    i+=1
    print(i)

print('done')

3111
done


In [18]:





max = len(tweets_list) -1
if i != len(tweets_list)-1:
    max = i - 1
print(len(tweets_list))
max

3111


3110

In [19]:
dick = {"name":user_name[:max],
        "age":age[:max],
        'location':location[:max],
        "gender":gender[:max],
        'join_date':joined[:max],
        'total tweets': tweet_count_user[:max],
        "followers":followers[:max],
        "following":following[:max],
        'verified' : verified[:max],
        'tweet':tweet[:max],
        'likes':likes[:max],
        'retweets':retweet_count[:max],
        'date':date[:max],
        'word count': word_count[:max],
        'launguage': lang[:max],
        'is quote':is_quote_status[:max],
        'trending': trending_for_area[:max]
       }

df = pd.DataFrame(dick)

In [20]:
df

,name,age,location,gender,join_date,total tweets,followers,following,verified,tweet,likes,retweets,date,word count,launguage,is quote,trending
0,arrowbluesky,NaN,None,None,2020-12-11,28823,794,811,False,"RT @btslyricistt: cypher pt.4, bts https://t.c...",0,1850,2022-12-30,6,en,False,None
1,storybehindme,NaN,None,None,2015-01-02,37003,328,201,False,"Okay but first, breakfast muna.",0,0,2022-12-30,5,en,False,None
2,glossysoul24,NaN,None,None,2021-09-17,3702,1,22,False,RT @Skullgirls: After 10 years of fighting the...,0,6413,2022-12-30,19,en,False,None
3,DiannaFWilliam1,NaN,None,None,2022-10-28,513,41,77,False,RT @TheBabylonBee: Another Actress Accuses Kir...,0,1314,2022-12-30,12,en,False,None
4,7milesounds,NaN,None,None,2020-04-26,44245,8,12,False,Disclosure - Happening,0,0,2022-12-30,3,en,False,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3105,tiredoframen,NaN,Colorado,None,2010-09-26,30992,388,693,False,RT @TimWeisberg: Due to an unforeseen issue wi...,0,1,2022-12-30,23,en,False,None
3106,tae_v_21,NaN,None,None,2012-12-28,279125,32,543,False,RT @jimanfc1306: 221230 | WEVERSE\n\nHappy Bir...,0,7336,2022-12-30,8,en,False,None
3107,Shard2xx,NaN,None,None,2016-03-25,1278,215,57,False,Women need to buy lashes to mimic a FRACTION o...,0,0,2022-12-30,12,en,False,None
3108,aji_nyam,NaN,None,None,2022-03-16,220,40,75,False,RT @brutusmarket: @SaharaReporters #MBuhari #P...,0,4,2022-12-30,17,en,False,None


In [21]:
DRIVER.quit()

In [22]:
df.to_csv('HOLY_SHIT_IT_WORKS.csv')